In [15]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
_ = load_dotenv()
api_key = os.environ["NVIDIA_API_KEY"]


def get_llm_output(temperature, max_tokens, system_role, prompt,
                   frequency_penalty=0, presence_penalty=0, stop=None):
    """
    Gọi LLM và trả về nội dung response. Nếu system_role là chuỗi rỗng,
    ta chỉ đưa message user, không đưa message system.
    """
    start_time = time.time()
    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key=api_key,
    )
    # Chỉ thêm system message khi system_role không rỗng
    messages = []
    if system_role and system_role.strip():
        messages.append({"role": "system", "content": system_role})
    messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
    )
    end_time = time.time()
    print('Finish generating response in', round((end_time - start_time), 2), 'seconds')
    return completion.choices[0].message.content

In [3]:
import pymupdf


def extract_text_from_pdfs(files):
    """Extract full text from a list of PDF files."""
    text = ""
    for f in files or []:
        with pymupdf.open(f) as pdf:
            text += "".join(page.get_text() for page in pdf)
    return text

In [7]:
text = extract_text_from_pdfs(['sample_resume.pdf'])

In [ ]:
def structure_cv(cv_text):
    system_prompt = """
You are a professional resume/CV formatter. 
Reformat the candidate's resume text into a clean, modern structure with these sections:
1. Header: Name, Contact Info
2. Summary: 2–3 sentence career summary
3. Experience: List of jobs with role, company, dates, bullet achievements
4. Education: Degrees, institutions, years
5. Projects (if any): title, description, technologies used
6. Skills: grouped by category (e.g., Languages, Tools, Frameworks)
7. Certifications (if any)

Output the resume in Markdown format. Use consistent bullet points and headers.
"""
    # Chia nhỏ nếu CV quá dài (ở đây giả định cv_text < 4000 tokens)
    prompt = f"""
Raw Resume Text:
\"\"\"
{cv_text}
\"\"\"

Please reformat as described.
"""
    return get_llm_output(
        temperature=0.3,
        max_tokens=1500,
        system_role=system_prompt,
        prompt=prompt
    )

In [17]:
aftext = structure_cv(text)

Finish generating response in 7.64 seconds


In [18]:
print(aftext)

### Header
FARAH MARTIN  
Data Analyst  
farahmartin@email.com | (123) 456-7890 | Brooklyn, NY | [LinkedIn](https://www.linkedin.com/in/farahmartin)

### Summary
Highly motivated and experienced data analyst with a strong background in mathematics and economics, leveraging technical skills to drive business growth and improvement. Proven track record of successfully implementing data-driven solutions to enhance operational efficiency, customer value, and revenue. Skilled in leading cross-functional teams and collaborating with executive teams to formulate and report on key performance indicators.

### Experience
* **Data Analyst, Fountain House**  
  May 2018 - Current, New York, NY
  * Built out the data and reporting infrastructure from the ground up using Tableau and SQL to provide real-time insights into the product, marketing funnels, and business KPIs.
  * Designed and implemented A/B experiments for products to improve the conversion rate by 19 basis points and reduce churn by 1

In [19]:
def structure_jd(jd_text):
    system_prompt = """
You are a professional job description formatter. 
Reformat the candidate's job description text into a clean, modern structure with these sections. Leave 'no information' for any section that is not applicable:
1. Header: Job Title, Company Name
2. Summary: 2–3 sentence overview of the role
3. Responsibilities: List of key responsibilities and tasks
4. Requirements: Skills and qualifications needed
5. Preferred Qualifications: Additional skills that are a plus
6. Benefits: Perks and benefits offered
7. Application Process: How to apply

Output the job description in Markdown format. Use consistent bullet points and headers.
"""
    # Chia nhỏ nếu JD quá dài (ở đây giả định jd_text < 4000 tokens)
    prompt = f"""
Raw Job Description Text:
\"\"\"
{jd_text}
\"\"\"

Please reformat as described.
"""
    return get_llm_output(
        temperature=0.3,
        max_tokens=1200,
        system_role=system_prompt,
        prompt=prompt
    )

In [20]:
jd = extract_text_from_pdfs(['sample_jd.pdf'])
afjdtext = structure_jd(jd)
print(afjdtext)


Finish generating response in 5.58 seconds
### Header: Data Analyst, [No Company Name]
### Summary
We are seeking a talented and detail-oriented Data Analyst to join our dynamic team, responsible for interpreting data, analyzing results, and providing actionable insights to drive informed decision-making. The Data Analyst will work closely with stakeholders to understand their data needs and present findings in a clear and concise manner. This role requires a strong analytical mindset and excellent communication skills.

### Responsibilities
* Extract, transform, and load (ETL) data from various sources
* Clean and preprocess data to ensure accuracy and consistency
* Develop scripts and workflows to automate data collection and processing tasks
* Perform exploratory data analysis to uncover trends, patterns, and anomalies
* Apply statistical and analytical techniques to derive insights from complex datasets
* Conduct hypothesis testing and predictive modeling to support business object

In [21]:
def generate_feedback(cv_text, jd_text):
    """
    Use LLM to generate structured, comprehensive feedback on the resume:
    - Đưa ra các tiêu chí đánh giá (Assessment Criteria)
    - Đánh giá từng mục (Review) dựa trên tiêu chí
    - Gợi ý cải tiến (Improvement Suggestions) để tăng tỉ lệ đậu
    """
    system_prompt = """
You are a senior career advisor and HR specialist. 
Your task is to evaluate the candidate’s resume against the Job Description, 
using the following structured framework:

1. Assessment Criteria:
   a. Relevance of Experience – How well past roles map to required responsibilities.
   b. Skills Match – Presence and prominence of key technical and soft skills.
   c. Projects - Relevance and impact of projects related to the JD.
   d. Achievement Impact – Use of quantifiable results and action verbs.
   e. Keywords & ATS Optimization – Inclusion of terminology from the JD.
   f. Clarity & Readability – Logical sectioning, bullet usage, concise language.
   g. Professional Branding – Strong summary, consistent formatting, contact info.

2. For each criterion:
   - Provide a **brief review**: highlight strengths and gaps.
   - Offer **2–3 short actionable suggestions** to improve that aspect of the resume.

3. Finally, summarize **top 3 priority actions** the candidate should take 
   to significantly increase their chance of being shortlisted.

Return your output in Markdown, with sections:
## Assessment
- **Relevance of Experience**  
  - Review: …  
  - Suggestions:
    1. …
    2. …

(Repeat for each criterion)

## Top 3 Priority Actions
1. …
2. …
3. …
"""

    user_prompt = f"""
Job Description:
\"\"\"
{jd_text}
\"\"\"

Candidate Resume:
\"\"\"
{cv_text}
\"\"\"
"""

    return get_llm_output(
        temperature=0.3,
        max_tokens=1200,
        system_role=system_prompt,
        prompt=user_prompt
    )

In [22]:
generated_feedback = generate_feedback(aftext, afjdtext)

Finish generating response in 15.14 seconds


In [23]:
print(generated_feedback)

## Assessment
### Relevance of Experience
- Review: The candidate's experience is highly relevant to the job description, with a strong background in data analysis, business intelligence, and related roles. The candidate has worked in various industries, including insurance and non-profit, and has experience in leading cross-functional teams and collaborating with executive teams.
- Suggestions:
  1. Emphasize the specific skills and tools used in previous roles, such as SQL, Python, and Tableau, to match the job requirements.
  2. Quantify the impact of previous projects, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn, to demonstrate the value added to previous employers.
  3. Consider adding a brief description of the company and industry for each previous role to provide context for the accomplishments.

### Skills Match
- Review: The candidate's skills match the job requirements, with proficiency in data analysis tools and programm

## Assessment
### Relevance of Experience
- Review: The candidate's experience is highly relevant to the job description, with a strong background in data analysis, business intelligence, and related roles. The candidate has worked in various industries, including insurance and non-profit, and has experience in leading cross-functional teams and collaborating with executive teams.
- Suggestions:
  1. Emphasize the specific skills and tools used in previous roles, such as SQL, Python, and Tableau, to match the job requirements.
  2. Quantify the impact of previous projects, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn, to demonstrate the value added to previous employers.
  3. Consider adding a brief description of the company and industry for each previous role to provide context for the accomplishments.

### Skills Match
- Review: The candidate's skills match the job requirements, with proficiency in data analysis tools and programming languages, such as SQL, Python, and Tableau. However, the candidate could benefit from highlighting additional skills, such as machine learning and data mining techniques.
- Suggestions:
  1. Expand the skills section to include additional technical skills, such as R, Julia, or other programming languages relevant to the job.
  2. Emphasize the candidate's experience with data visualization tools, such as Tableau, and highlight any certifications or training in these areas.
  3. Consider adding a section for "soft skills" to highlight the candidate's excellent communication and presentation abilities.

### Projects
- Review: The candidate's projects and accomplishments are highly relevant to the job description, with a focus on data-driven solutions, operational efficiency, and revenue growth. However, the candidate could benefit from providing more details about the projects, such as the methodologies used and the impact on the business.
- Suggestions:
  1. Provide more details about the projects, such as the goals, methodologies, and outcomes, to demonstrate the candidate's expertise and value added.
  2. Consider adding a separate section for "projects" or "case studies" to showcase the candidate's accomplishments and provide a clear overview of their experience.
  3. Emphasize the candidate's ability to work independently and collaboratively on projects, and highlight any experience with agile methodologies or project management tools.

### Achievement Impact
- Review: The candidate's achievements are quantified, with specific metrics and outcomes, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn. However, the candidate could benefit from using more action verbs and emphasizing the impact on the business.
- Suggestions:
  1. Use more action verbs, such as "Improved," "Increased," and "Enhanced," to describe the candidate's achievements and emphasize the impact on the business.
  2. Quantify the impact of the candidate's achievements, such as the revenue growth or cost savings, to demonstrate the value added to previous employers.
  3. Consider adding a section for "achievements" or "accomplishments" to showcase the candidate's impact and provide a clear overview of their experience.

### Keywords & ATS Optimization
- Review: The candidate's resume includes relevant keywords, such as "data analysis," "business intelligence," and "SQL," but could benefit from additional keywords and phrases to match the job requirements.
- Suggestions:
  1. Incorporate additional keywords and phrases from the job description, such as "machine learning," "data mining," and "data visualization," to improve the candidate's visibility in applicant tracking systems (ATS).
  2. Use a keyword cloud or tag cloud to visualize the candidate's skills and expertise, and provide a clear overview of their experience.
  3. Consider adding a section for "technical skills" or "tools" to showcase the candidate's proficiency in specific software and programming languages.

### Clarity & Readability
- Review: The candidate's resume is well-organized, with clear section headings and concise bullet points. However, the candidate could benefit from using more white space and emphasizing the most important information.
- Suggestions:
  1. Use more white space to improve the readability of the resume, and emphasize the most important information, such as the candidate's achievements and skills.
  2. Consider using a more modern font, such as Arial or Calibri, to improve the overall appearance of the resume.
  3. Use bullet points and action verbs to describe the candidate's experience and achievements, and provide a clear overview of their skills and expertise.

### Professional Branding
- Review: The candidate's professional branding is strong, with a clear and concise summary, consistent formatting, and a professional online presence.
- Suggestions:
  1. Consider adding a professional summary or career objective statement to the top of the resume, to provide a clear overview of the candidate's experience and goals.
  2. Use a consistent format throughout the resume, with clear section headings and concise bullet points, to improve the overall appearance and readability.
  3. Emphasize the candidate's unique value proposition, such as their experience in leading cross-functional teams or collaborating with executive teams, to differentiate themselves from other candidates.

## Top 3 Priority Actions
1. **Emphasize relevant technical skills**: Incorporate additional keywords and phrases from the job description, such as "machine learning," "data mining," and "data visualization," to improve the candidate's visibility in applicant tracking systems (ATS) and match the job requirements.
2. **Quantify achievements and impact**: Use more action verbs and emphasize the impact on the business, such as the revenue growth or cost savings, to demonstrate the value added to previous employers and provide a clear overview of the candidate's experience.
3. **Use a more modern and concise format**: Use more white space, a modern font, and concise bullet points to improve the readability and overall appearance

In [ ]:
import os
import gradio as gr
import fitz  # PyMuPDF for PDF parsing
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI

# === Configuration ===
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Directory to persist Chroma vector DBs
PERSIST_DIR = "./chroma_db"

# Initialize embedding model and LLM
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
llm = OpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4")

# Text splitter for chunking documents
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def extract_text_from_pdfs(files):
    """Extract full text from a list of PDF files."""
    text = ""
    for f in files or []:
        pdf = fitz.open(stream=f.read(), filetype="pdf")
        for page in pdf:
            text += page.get_text()
    return text

def chunk_and_index(text, collection_name):
    """Chunk text and index into Chroma under a specific collection."""
    docs = splitter.split_text(text)
    vectordb = Chroma(
        collection_name=collection_name,
        embedding_function=embeddings,
        persist_directory=PERSIST_DIR
    )
    # Add texts with metadata
    vectordb.add_texts(texts=docs, metadatas=[{"source": collection_name}] * len(docs))
    vectordb.persist()

def generate_feedback(cv_text, jd_text):
    """Use LLM to generate resume improvement suggestions."""
    prompt = f"""
You are a skilled career advisor. 
Given the following Job Description and the Candidate's Resume, 
provide actionable bullet-point feedback on how to improve the resume 
to better match the job requirements.

Job Description:
\"\"\"
{jd_text}
\"\"\"

Candidate Resume:
\"\"\"
{cv_text}
\"\"\"

Please list at least 5 suggestions.
"""
    return llm(prompt)

def handle_upload(cv_files, jd_files):
    """Main handler: extract, index, and feedback."""
    if not cv_files or not jd_files:
        return "❌ Please upload both CV and Job Description PDFs."
    # Extract texts
    cv_text = extract_text_from_pdfs(cv_files)
    jd_text = extract_text_from_pdfs(jd_files)
    # Index into RAG vector DB
    chunk_and_index(cv_text, "cv_collection")
    chunk_and_index(jd_text, "jd_collection")
    # Generate feedback
    feedback = generate_feedback(cv_text, jd_text)
    return feedback

# === Gradio UI ===
with gr.Blocks(title="RAG CV-JD Analyzer") as demo:
    gr.Markdown("## Upload CV & Job Description")
    with gr.Row():
        cv_upload = gr.File(label="Candidate CV (PDF)", file_types=[".pdf"], file_count="multiple")
        jd_upload = gr.File(label="Job Description (PDF)", file_types=[".pdf"], file_count="multiple")
    analyze_btn = gr.Button("Analyze & Feedback")
    feedback_box = gr.Textbox(label="Improvement Suggestions", lines=10, interactive=False)

    analyze_btn.click(
        fn=handle_upload,
        inputs=[cv_upload, jd_upload],
        outputs=feedback_box
    )

demo.launch()
